# KarpovCourses RecSys by Vladimir Polukhin
### Добро пожаловать, меня зовут Владимир. Данный проект создан в рамках прохождения курса Start ML после заверешения изучения первых двух модулей, которые захватывают такие темы, как 'Прикладная разработка на Python' и 'Классическое машинное обучение и приложения'. 
### Перед просмотром данного ноутбука следует заглянуть в data_exploring.ipynb, чтобы понять, почему именно такие действия я совершаю с данными
##### Для полного запуска ноутбука рекомендуется не мнее 32gb RAM!!!

# Необходимые библиотеки
##### См. requirements.txt

In [ ]:
import os

import numpy as np
import pandas as pd
import seaborn as sns
import texthero as hero
from sklearn.model_selection import train_test_split
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay, accuracy_score
from catboost import CatBoostClassifier

# База Данных

In [ ]:
database = os.getenv('POSTGRES_KEY')

# Обработка user_data

##### age - Возраст пользователя (в профиле)
##### city - Город пользователя (в профиле)
##### country - Страна пользователя (в профиле)
##### exp_group - Экспериментальная группа: некоторая зашифрованная категория
##### gender - Пол пользователя
##### id - Уникальный идентификатор пользователя
##### os - Операционная система устройства, с которого происходит пользование соц.сетью
##### source - Пришел ли пользователь в приложение с органического трафика или с рекламы

In [ ]:
user_data = pd.read_sql(
    'SELECT * FROM user_data',
    database
)

users_like_rate = pd.read_sql(
    '''
    SELECT user_id, SUM(target), COUNT(target)
    FROM feed_data
    GROUP BY user_id
    ''',
    database
)

In [ ]:
user_data

In [ ]:
user_data = user_data.drop(['os', 'source', 'gender', 'exp_group'], axis=1)

In [ ]:
user_data = pd.concat((user_data, pd.DataFrame(users_like_rate['sum'] / users_like_rate['count'], columns=['user_like_rate'])), axis=1)

# Обработка post_text

##### id - Уникальный идентификатор поста
##### text - Текстовое содержание поста
##### topic - Основная тематика

In [ ]:
post_text = pd.read_sql(
    'SELECT * FROM post_text_df',
    database
)

posts_like_rate = pd.read_sql(
    '''
    SELECT post_id, SUM(target), COUNT(target)
    FROM feed_data
    GROUP BY post_id
    ''',
    database
)

In [ ]:
post_text

In [ ]:
post_text['text'] = hero.clean(post_text['text'])
tfidf_text = hero.tfidf(post_text['text'])

In [ ]:
tfidf_text_mean_list = []

for tf_idf in tfidf_text:
    tfidf_text_mean_list.append(np.mean(tf_idf))

post_text['text'] = pd.DataFrame(tfidf_text_mean_list, columns=['tfidf'])

In [ ]:
post_text = pd.concat((post_text.drop('topic', axis=1), pd.get_dummies(post_text['topic'])), axis=1)

In [ ]:
posts_like_rate = pd.concat((posts_like_rate.drop(['sum', 'count'], axis=1), pd.DataFrame(posts_like_rate['sum'] / posts_like_rate['count'], columns=['post_like_rate'])), axis=1)

In [ ]:
temporary_post_text = pd.merge(post_text, posts_like_rate, on='post_id')

In [ ]:
post_text = temporary_post_text[temporary_post_text['post_like_rate'] > 0.095418]

# Загрузка post_feed

##### timestamp - Время, когда был произведен просмотр
##### user_id - id пользователя, который совершил просмотр
##### post_id - id просмотренного поста
##### action - Тип действия: просмотр или лайк
##### target - 1 у просмотров, если почти сразу после просмотра был совершен лайк, иначе 0. У действий like пропущенное значение.

In [ ]:
feed_data = pd.read_sql(
    '''
    SELECT user_id, post_id, target
    FROM feed_data
    WHERE action = 'view'
    ORDER BY timestamp
    ''',
    database
)

# Соединение таблиц в один датасет

In [ ]:
temporary_data = pd.merge(feed_data, user_data, on='user_id')
data = pd.merge(temporary_data, post_text, on='post_id')

In [ ]:
data

In [ ]:
data['city'] = data.groupby('city')['target'].transform('mean')
data['country'] = data.groupby('country')['target'].transform('mean')
data['topic'] = data.groupby('topic')['target'].transform('mean')

##### Данный порог установлен для того, чтобы сократить количество записей в датасете, при этом оставив информацию о каждом юзере

In [ ]:
temporary_data = data[data['post_like_rate'] > 0.13347]
data = temporary_data[temporary_data['text'] > 0.0000636]

# Обучение модели

In [ ]:
data['target'].value_counts()

##### Неравномерное распределение таргета -> нужно использовать сбалансированные веса в моделе

In [ ]:
X = data.drop('target', axis=1)
y = data['target']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, shuffle=True)

In [ ]:
model = CatBoostClassifier(auto_class_weights='Balanced', depth=6, l2_leaf_reg=0.2, early_stopping_rounds=15, learning_rate=0.5, iterations=2000, random_state=1)

In [ ]:
model.fit(X_train, y_train)

# Замер ROC-AUC, PR-AUC и accuracy

In [ ]:
RocCurveDisplay.from_estimator(model, X_train, y_train)

In [ ]:
PrecisionRecallDisplay.from_estimator(model, X_train, y_train)

In [ ]:
round(accuracy_score(y_test, model.predict(X_test)), 4), round(accuracy_score(y_test, model.predict(X_train)), 4)

##### Также стоит отметить, что основной метрикой проекта является hitrate:
##### Она принимает значение 1, если среди предложенных 5 рекомендаций хотя бы 1 получила в итоге like от пользователя. Даже если все 5 предложенных постов в итоге будут оценены пользователем, все равно hitrate будет равен 1. Метрика бинарная! В противном случае, если ни один из предложенных постов не был оценен пользователем, hitrate  принимает значение 0. 

# Сохранение модели

In [ ]:
model.save_model('balanced_model', 
                 format="cbm")